**Constitution des jeux de données pour le Challenge de la Data party**

*Les 30 ans de la MTD*

# Les jeux de données TETIS publiés

Champs d'intérêt:

- Localisation de la production : latestVersion.metadataBlocks.geospatial.fields

In [3]:
import configparser
import requests
import pandas as pd
import numpy as np

In [4]:
credential_file = "./../credentials.ini"
credential_config = configparser.ConfigParser()
credential_config.read(credential_file)
print(credential_config.sections())

['DATA_INRAE', 'PREPROD_DATA_INRAE', 'CIRAD']


In [5]:
# Paramétrage API data.inrae.fr
API_TOKEN_data_inrae = credential_config['DATA_INRAE']['API_TOKEN']
url_rdg = "https://entrepot.recherche.data.gouv.fr/"
headers_data_inrae = {'Accept': 'application/json',
                     'X-Dataverse-key': API_TOKEN_data_inrae
}

# Paramétrage API https://dataverse.cirad.fr/
API_TOKEN_data_cirad = credential_config['CIRAD']['API_TOKEN']
url_cirad = "https://dataverse.cirad.fr/"
headers_data_cirad = {'Accept': 'application/json',
                     'X-Dataverse-key': API_TOKEN_data_cirad
}

In [6]:
# get the total size of a dataverse
def storage_szie_of_dataverse(dataverse, url=url_rdg, header=headers_data_inrae):
    url_dataverse = url + "api/dataverses/" + dataverse
    reponse = requests.get(url_dataverse + "/storagesize", headers=header).json()["data"]
    return reponse

# get dataset ID from a dataverse
def datasetID_from_dataverse(dataverse, url=url_rdg, header=headers_data_inrae):
    url_dataverse = url + "/api/dataverses/" + dataverse
    try:
        reponse = requests.get(url_dataverse + "/contents", headers=header).json()["data"]
    except:
        print(requests.get(url_dataverse + "/contents", headers=header).json())
        reponse = {}
    return reponse

# get all data from a dataverse
def datasets_info_from_datasetID(dataverse_content, url=url_rdg, header=headers_data_inrae):
    """
    """
    list_datasets = []
    list_dataverses = []
    for dataset in dataverse_content:
        try: 
            dataset_info = requests.get(url + "/api/datasets/" + str(dataset["id"]), headers=header).json()["data"]
            list_datasets.append(dataset_info)
        except: # it's not a dataset but a dataverse
            list_dataverses.append(str(dataset["title"]).replace(" ", "_"))
    return list_datasets, list_dataverses

# get view (total and uniques), download (unique & totla) and citation of a dataset
def get_metrics_of_datasets(doi):
    api_metrics = ["viewsTotal", "viewsUnique", "downloadsTotal", "downloadsUnique", "citations"]
    dic_api_metrics = {}
    for metric in api_metrics:
        url = url_rdg + "/api/datasets/:persistentId/makeDataCount/" + metric + "?persistentId=" + doi
        try: 
            reponse = requests.get(url, headers=headers_data_inrae).json()["data"][metric]
        except:
            reponse = 0
        dic_api_metrics[metric] = reponse
    print(dic_api_metrics)
    return dic_api_metrics

def get_metrics_of_datasets_cirad(doi):
    api_metrics = ["viewsTotal", "viewsUnique", "downloadsTotal", "downloadsUnique", "citations"]
    dic_api_metrics = {}
    for metric in api_metrics:
        url = url_cirad + "/api/datasets/:persistentId/makeDataCount/" + metric + "?persistentId=" + doi
        try: 
            reponse = requests.get(url, headers=headers_data_cirad).json()["data"][metric]
        except:
            reponse = 0
        dic_api_metrics[metric] = reponse
    return dic_api_metrics

In [ ]:
list_of_dataverse_to_explore = []
list_of_dataverse_to_explore.append("tetis")
list_datasets = []
# Jump into dataverses inside the dataverse of TETIS!
while list_of_dataverse_to_explore:
    for dataverse in list_of_dataverse_to_explore:
        print("RDG: " + dataverse)
        datasetIDs = datasetID_from_dataverse(dataverse)
        print("\t Number of objects: " + str(len(datasetIDs)))
        try:
            list_datasets_local, list_dataverses_local = datasets_info_from_datasetID(datasetIDs)
            list_datasets.extend(list_datasets_local)
            list_of_dataverse_to_explore.extend(list_dataverses_local)
            print("\t Number of datasets: " + str(len(list_datasets_local)))
            print("\t Number of dataverse: " + str(len(list_dataverses_local)))
        except:
            pass

        print("Dataverse CIRAD: " + dataverse)
        datasetIDs = datasetID_from_dataverse(dataverse, url=url_cirad, header=headers_data_cirad)
        print("\t Number of objects: " + str(len(datasetIDs)))
        try:
            list_datasets_local, list_dataverses_local = datasets_info_from_datasetID(datasetIDs, url=url_cirad, header=headers_data_cirad)
            list_datasets.extend(list_datasets_local)
            list_of_dataverse_to_explore.extend(list_dataverses_local)
            print("\t Number of datasets: " + str(len(list_datasets_local)))
            print("\t Number of dataverse: " + str(len(list_dataverses_local)))
        except:
            pass

        list_of_dataverse_to_explore.remove(dataverse)
df_datasets = pd.json_normalize(list_datasets, max_level=10)